<a href="https://colab.research.google.com/github/ywchanna2001/Pet_Care_App_BE/blob/main/Video_Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install SDK

In [2]:
%pip install -U -q 'google-genai'

## Setup API key

In [3]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

## Initialize SDK client

In [4]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
print(type(client))

<class 'google.genai.client.Client'>


## Select the Gemini model

Video understanding works best Gemini 2.5 pro model. We can also select former models to compare their behavior but it is recommended to use at least the 2.0 ones.

Official comparison:
https://deepmind.google/technologies/gemini/pro/

In [6]:
model_name = "gemini-2.5-pro-exp-03-25" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

## Analyze youtube videos

Until we create a rabies dog video data set we decided to analyze and process the videos that include rabid dogs' visuals.

In [15]:
import google.genai as genai
from google.genai import types
import json
from PIL import Image
from IPython.display import display, Markdown, HTML

def analyze_youtube_video(prompt: str, youtube_url: str, client: genai.Client, model: str) -> str:
    """
    Analyzes a YouTube video using the Gemini API with a custom prompt.

    Args:
        prompt: The text prompt/instructions for the AI model.
        youtube_url: The URL of the YouTube video to analyze.
        client: The initialized google.genai.Client instance.
        model: The name of the Gemini model to use (e.g., "gemini-1.5-pro-preview-latest").

    Returns:
        The text response generated by the Gemini model, or an error message.

    Raises:
        # Catches general exceptions, specific API errors might be handled differently.
        Exception: If the API call fails for reasons other than generation blocking.
    """
    print(f"Attempting to analyze YouTube video: {youtube_url}")
    print(f"Using Model: {model}")
    print(f"Prompt: '{prompt}'")

    try:
        # Create the video part using FileData with the URI
        video_part = types.Part(
            file_data=types.FileData(file_uri=youtube_url)
        )

        # Create the text prompt part
        prompt_part = types.Part(text=prompt)

        # Construct the content object with both parts
        # The order of parts generally doesn't matter, but prompt first is common
        content_to_send = types.Content(parts=[prompt_part, video_part])

        # Make the API call
        response = client.models.generate_content(
            model=model,
            contents=content_to_send
        )

        # Check if the response has text (it might be blocked)
        if hasattr(response, 'text'):
             print("Analysis successful.")
             return response.text
        else:
             # Handle cases where the response might be empty or blocked
             print("Warning: Response did not contain text. It might have been blocked.")
             # You might want to inspect response.prompt_feedback or other attributes
             return f"Analysis generated no text output. Response details: {response}"


    except Exception as e:
        # Handle potential errors during the API call
        print(f"An error occurred during Gemini API call: {e}")
        # You might want to log the full exception traceback here
        # import traceback
        # traceback.print_exc()
        return f"Error analyzing video: {e}"

# --- Example Usage ---

# **Important:** Make sure 'client' and 'model_name' are defined and valid before running this.
if 'client' in locals() and 'model_name' in locals():

    # Define your custom prompt and video URL
    my_custom_prompt = """
**Role:** You are an AI assistant specialized in analyzing animal behavior from video footage. You are tasked with identifying behavioral signs potentially associated with neurological conditions like rabies in dogs, based *only* on the visual information provided. You are NOT a veterinarian and cannot provide a diagnosis.

**Task:** Analyze the provided video clip of a dog. Identify any behaviors exhibited that are commonly listed as potential symptoms of rabies. Based *solely* on the observed behaviors in the video, assess a *potential risk level* (Low, Medium, High) that these behaviors *could be consistent* with rabies symptoms. Provide a concise explanation for your assessment, referencing specific visual evidence.

**Input:** A video clip showing a dog's behavior.

**Analysis Guidelines:**
1.  **Observe Behaviors:** Carefully watch the dog's actions, movements, posture, and any audible sounds if discernible.
2.  **Identify Potential Signs:** Look specifically for behaviors such as:
    *   Unprovoked aggression or unusual irritability
    *   Excessive drooling or foaming at the mouth (hypersalivation)
    *   Difficulty swallowing (dysphagia)
    *   Strange vocalizations (unusual barking, howling, whining)
    *   Stumbling, incoordination, weakness, or paralysis (especially in the hind legs)
    *   Disorientation, confusion, or vacant staring
    *   Restlessness, pacing, or agitation
    *   Seizures
    *   Jaw dropping or facial paralysis
    *   Aversion to water (hydrophobia - rare to capture clearly on video but note any related behavior)
    *   Self-mutilation (biting/chewing at limbs)
3.  **Assess Risk Level:** Based on the presence, combination, and severity of the *observed signs only*:
    *   **Low Risk:** No specific rabies-associated signs observed, or behavior appears normal/explainable by context (e.g., playful barking, normal tiredness).
    *   **Medium Risk:** One or two potential signs observed, but they could also have other causes (e.g., some agitation, slight incoordination that could be an injury). The behavior warrants caution.
    *   **High Risk:** Multiple significant signs strongly associated with rabies are clearly visible (e.g., unprovoked aggression combined with facial paralysis and hypersalivation; severe disorientation with seizures).
4.  **Explain Your Reasoning:** Briefly describe *which specific behaviors* (or lack thereof) led to your risk assessment. Be factual and refer only to what is visible in the video.
5.  **Mandatory Disclaimer:** ALWAYS conclude your explanation with: "Disclaimer: This analysis is based solely on the provided video and is NOT a veterinary diagnosis. Rabies is a serious condition. If you have any concerns about this animal's health or behavior, consult a qualified veterinarian immediately."

**Output Format:** Please provide the response in JSON format with the following keys:
*   `risk_level`: (String: "Low", "Medium", or "High")
*   `explanation`: (String: Your detailed explanation, including the mandatory disclaimer)
*   `observed_signs`: (List of Strings: Specific potential signs observed, e.g., ["Unprovoked aggression", "Hypersalivation"])

**Example Video Input:** [Video Data or Reference to Video Data will be provided here by the API call]
"""


    video_to_analyze_url = 'https://youtube.com/shorts/KtQM7kHVDxA?si=OuN01NKsm2ANto2T'

    # Call the function
    analysis_result = analyze_youtube_video(
        prompt=my_custom_prompt,
        youtube_url=video_to_analyze_url,
        client=client,
        model=model_name
    )

    # Display the result using Markdown
    print("\n--- Analysis Result ---")
    if analysis_result.startswith("Error"):
        print(analysis_result) # Print error messages directly
    else:
        display(Markdown(analysis_result))

else:
    print("Error: Please ensure 'client' (genai.Client) and 'model_name' are defined.")

Attempting to analyze YouTube video: https://youtube.com/shorts/KtQM7kHVDxA?si=OuN01NKsm2ANto2T
Using Model: gemini-2.5-pro-exp-03-25
Prompt: '
**Role:** You are an AI assistant specialized in analyzing animal behavior from video footage. You are tasked with identifying behavioral signs potentially associated with neurological conditions like rabies in dogs, based *only* on the visual information provided. You are NOT a veterinarian and cannot provide a diagnosis.

**Task:** Analyze the provided video clip of a dog. Identify any behaviors exhibited that are commonly listed as potential symptoms of rabies. Based *solely* on the observed behaviors in the video, assess a *potential risk level* (Low, Medium, High) that these behaviors *could be consistent* with rabies symptoms. Provide a concise explanation for your assessment, referencing specific visual evidence.

**Input:** A video clip showing a dog's behavior.

**Analysis Guidelines:**
1.  **Observe Behaviors:** Carefully watch the do

```json
{
  "risk_level": "Low",
  "explanation": "The dog in the video displays calm and tolerant behavior while interacting with a person applying a slap bracelet. There are no observable signs commonly associated with rabies, such as unprovoked aggression, excessive drooling, disorientation, incoordination, facial paralysis, unusual vocalizations, or seizures. The dog sits patiently and does not react negatively to the stimulus. Disclaimer: This analysis is based solely on the provided video and is NOT a veterinary diagnosis. Rabies is a serious condition. If you have any concerns about this animal's health or behavior, consult a qualified veterinarian immediately.",
  "observed_signs": []
}
```